![Redis](https://redis.com/wp-content/themes/wpx/assets/images/logo-redis.svg?auto=webp&quality=85,75&width=120)
# RAG with LangChain

This notebook uses [LangChain](https://python.langchain.com/docs/get_started/introduction) and [Redis](https://redis.com) to perform document + embdding indexing and semantic search tasks. It also shows how to integrate with an LLM like OpenAI's GPT models.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/financial-vss/blob/main/langchain-03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment Setup

### Pull Github Materials
Because you are likely running this notebook in **Google Colab**, we need to first
pull the necessary dataset and materials directly from GitHub.

**If you are running this notebook locally**, FYI you may not need to perform this
step at all.

In [1]:
# This clones the supporting git repository into a directory named 'temp_repo'.
!git clone https://github.com/redis-developer/financial-vss.git temp_repo

# This command moves the 'resources' directory from 'temp_repo' to your current directory.
!mv temp_repo/resources .
!mv temp_repo/requirements.txt .

# This deletes the 'temp_repo' directory, cleaning up the unwanted files.
!rm -rf temp_repo

Cloning into 'temp_repo'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 81 (delta 35), reused 61 (delta 19), pack-reused 0
Receiving objects: 100% (81/81), 7.00 MiB | 24.97 MiB/s, done.
Resolving deltas: 100% (35/35), done.


### Install Python Dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
import warnings

warnings.filterwarnings("ignore")

### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings created from PDF document chunks. **We need to make sure we have a Redis
instance available.**

#### Method 1: localized Redis Stack
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly
from the Redis package archive.

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### Method 2: Redis Cloud
Instead of using the in-notebook, localized Redis Stack, you can quickly deploy a
[FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [ ]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

## RAG with LangChain

### Dataset Preparation (PDF Documents)

To best demonstrate Redis as a vector database layer, we will load a single
financial (10k filings) doc and preprocess it using some helpers from LangChain:

- `UnstructuredFileLoader` is not the only document loader type that LangChain provides. Docs: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file
- `RecursiveCharacterTextSplitter` is what we use to create smaller chunks of text from the doc. Docs: https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader

# Load list of pdfs from a folder
data_path = "resources/"
docs = [os.path.join(data_path, file) for file in os.listdir(data_path)]

print("Listing available documents ...", docs)

Listing available documents ... ['resources/nvd-10k-2023.pdf', 'resources/nke-10k-2023.pdf', 'resources/msft-10k-2023.pdf', 'resources/amzn-10k-2023.pdf', 'resources/jnj-10k-2023.pdf', 'resources/aapl-10k-2023.pdf']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Done preprocessing. Created 323 chunks of the original pdf resources/nke-10k-2023.pdf


In [ ]:
# pick out the Nike doc for this exercise
doc = [doc for doc in docs if "nke" in doc][0]

# set up the file loader/extractor and text splitter to create chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500, chunk_overlap=0
)
loader = UnstructuredFileLoader(
    doc, mode="single", strategy="fast"
)

# extract, load, and make chunks
chunks = loader.load_and_split(text_splitter)

print("Done preprocessing. Created", len(chunks), "chunks of the original pdf", doc)

### Initialize Embeddings Engine
Here we will use LangChain's built in embedding engine so that it will work seemlessly with the LangChain VectorStore classes.

In [7]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

## Vector Search with LangChain
### Create Redis vector store instance

We also need to create a schema for the vector index so we can take advantage of the metadata along with the vectors.

**Important Note**: LangChain does not support JSON data types yet. Only supports HASH for now. This update should be coming soon.

In [8]:
from langchain.vectorstores.redis import Redis


# set the index name for this example
index_name = "langchain"

# with langchain we can manually modify the default vector schema configuration
vector_schema = {
    "name": "chunk_vector",        # name of the vector field in langchain
    "algorithm": "HNSW",           # could use HNSW instead
    "dims": 384,                   # set based on the HF model embedding dimension
    "distance_metric": "COSINE",   # could use EUCLIDEAN or IP
    "datatype": "FLOAT32",
}

# here we can define the entire schema spec for our index in LangChain
index_schema = {
    "vector": [vector_schema],
    "text": [{"name": "content"}],
    "content_vector_key": "chunk_vector"    # name of the vector field in langchain
}


# construct the vector store class from texts and metadata
rds = Redis.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name,
    redis_url=REDIS_URL,
    index_schema=index_schema,
)

If you meant to manually override the schema, please ignore this message.
index_schema: {'vector': [{'name': 'chunk_vector', 'algorithm': 'HNSW', 'dims': 384, 'distance_metric': 'COSINE', 'datatype': 'FLOAT32'}], 'text': [{'name': 'content'}], 'content_vector_key': 'chunk_vector'}
generated_schema: {'text': [{'name': 'source'}], 'numeric': [{'name': 'start_index'}], 'tag': []}



In [9]:
# If you wish to connect to an existing Redis vector store instance
rds = Redis.from_existing_index(
    embedding=embeddings,
    index_name=index_name,
    schema=index_schema,
    redis_url=REDIS_URL,
)

In [10]:
# checkout out the schema we created
rds.schema

{'text': [{'name': 'content',
   'weight': 1,
   'no_stem': False,
   'withsuffixtrie': False,
   'no_index': False,
   'sortable': False}],
 'vector': [{'name': 'chunk_vector',
   'dims': 384,
   'algorithm': 'HNSW',
   'datatype': 'FLOAT32',
   'distance_metric': 'COSINE',
   'initial_cap': 20000,
   'm': 16,
   'ef_construction': 200,
   'ef_runtime': 10,
   'epsilon': 0.8}]}

In [11]:
# access underlying redis client to see how many docs have been stores
rds.client.dbsize()

323

In [12]:
# do NOT run this command in production
keys = rds.client.keys()

rds.client.hgetall(keys[0])

{b'start_index': b'146100',
 b'source': b'resources/nke-10k-2023.pdf',
 b'content': b"Through the Consumer Direct Acceleration strategy, we are focused on creating the marketplace of the future with more premium, consistent and seamless consumer experiences, leading with digital and our owned stores, as well as select wholesale partners. In addition, our product creation and marketing organizations are aligned to a consumer construct focused on sports dimensions through Men's, Women's and Kids', which allows us to better serve consumer needs. We continue to invest in a new Enterprise Resource Planning Platform, data and analytics, demand sensing, insight gathering, and other areas to create an end-to-end technology foundation, which we believe will further accelerate our digital transformation. We believe this unified approach will accelerate growth and unlock more efficiency for our business, while driving speed and responsiveness as we serve consumers globally.\n\nFINANCIAL HIGHLIGHT

### Query the database
Now we can use the LangChain vector store class to perform similarity search operations on Redis

In [13]:
from langchain.vectorstores.redis import RedisText

In [14]:
# basic "top 4" vector search on a given query
rds.similarity_search_with_score(query="Profit margins", k=4)

[(Document(page_content="increased 18%, driven by strong digital sales growth of 23%, comparable store sales growth of 9% and the addition of new stores.\n\nFootwear revenues increased 22% on a currency-neutral basis, primarily due to higher revenues in Men's and the Jordan Brand. Unit sales of footwear increased\n\n17%, while higher ASP per pair contributed approximately 5 percentage points of footwear revenue growth. Higher ASP per pair was primarily due to higher full-price ASP and growth in NIKE Direct, partially offset by lower NIKE Direct ASP, reflecting higher promotional activity as well as lower available inventory supply in the prior period and a lower mix of full-price sales.\n\nApparel revenues increased 9% on a currency-neutral basis, primarily due to higher revenues in Men's. Unit sales of apparel increased 7%, while higher ASP per unit contributed approximately 2 percentage points of apparel revenue growth. Higher ASP per unit was primarily due to higher full-price ASP a

In [15]:
# vector search with metadata filtering

f = RedisText("content") % "profit"
rds.similarity_search_with_score(query="Profit margins", k=4, filter=f)

[(Document(page_content='COMPARABLE STORE SALES Comparable store sales: This key metric, which excludes NIKE Brand Digital sales, comprises revenues from NIKE-owned in-line and factory stores for which all three of the following requirements have been met: (1) the store has been open at least one year, (2) square footage has not changed by more than 15% within the past year and (3) the store has not been permanently repositioned within the past year. Comparable store sales includes revenues from stores that were temporarily closed during the period as a result of COVID-19. Comparable store sales represents a performance metric that we believe is useful information for management and investors in understanding the performance of our established NIKE-owned in-line and factory stores. Management considers this metric when making financial and operating decisions. The method of calculating comparable store sales varies across the retail industry. As a result, our calculation of this metric

In [16]:
# vector search with combinations of metadata filtering

f = (RedisText("content") % "profit") | (RedisText("content") % "revenue")
rds.similarity_search_with_score(query="Nike company revenue", k=4, filter=f)

[(Document(page_content='FISCAL 2023 COMPARED TO FISCAL 2022\n\nNIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively. The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6, 2 and 1 percentage points to NIKE, Inc. Revenues, respectively.\n\nNIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This increase was primarily due to higher revenues in Men\'s, the Jordan Brand, Women\'s and Kids\' which grew 17%, 35%,11% and 10%, respectively, on a wholesale equivalent basis.\n\nNIKE Brand footwear revenues increased 20% on a currency-neutral basis, due to higher revenues in Men\'s, the Jordan Brand, Women\'s and Kids\'. Unit sales of footwear increased 13%, while higher average selling pr

In [17]:
# filter results to a certain distance threshold
rds.similarity_search_with_score(query="Nike company revenue", k=4, distance_threshold=0.14)

[]

## RAG with LangChain
LangChain makes it easy to now take this vector store and build retireval augmented generation (RAG) applications over your data.

### Initialize OpenAI

You need to supply an OpenAI API key (starts with `sk-...`) when prompted. If the key is in your env -- great, otherwise enter it when prompted below. You can find your API key at https://platform.openai.com/account/api-keys

In [18]:
import getpass
from langchain.llms import OpenAI

llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY") or getpass.getpass(prompt="OpenAI API Key:"))

OpenAI API Key:··········


### Setup prompt
PromptTemplate defines the exect text of the response that would be fed to the LLM. This step is optional, but the defaults usually work well for OpenAI and might fall short for other models.

In [19]:
def get_prompt():
    """Create the QA chain."""
    from langchain.prompts import PromptTemplate

    # Define our prompt
    prompt_template = """Use the following pieces of context from financial 10k filings data to answer the quser uestion at the end. If you don't know the answer, say that you don't know, don't try to make up an answer.

    This should be in the following format:

    Question: [question here]
    Answer: [answer here]

    Begin!

    Context:
    ---------
    {context}
    ---------
    Question: {question}
    Answer:"""

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    return prompt

### Putting it all together

This is where the Langchain brings all the components together in a form of a simple RAG application with the financial PDF document.

In [31]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=rds.as_retriever(search_type="similarity_distance_threshold",search_kwargs={"distance_threshold":0.5}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": get_prompt()},
    #verbose=True
)

### Finally - let's ask questions!



In [32]:
query = "What was Nike's revenue last year compared to this year??"
res=qa(query)
res['result']

' NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.'

In [33]:
query = "How many products does Nike offer? What is the industry that Nike is part of?"
res=qa(query)
res['result']

' Nike offers athletic footwear, apparel, equipment, accessories and services. Nike is part of the athletic footwear, apparel and equipment industry.'

In [34]:
query = "Is Nike an ethical company?"
res=qa(query)
res['result']

" I don't know."

In [35]:
query = "How many employees work at Nike???"
res=qa(query)
res['result']

' Approximately 83,700.'

## Cleanup

Cleanup the index and data.

In [36]:
#rds.drop_index(index_name=index_name, redis_url=REDIS_URL, delete_documents=True)